# Week 4: Machine learning 1

Machine learning is a technique for learning patterns in data that enable computers to make decisions and predictions. It's probably one of the hottest skills to master as a scientist or engineer in research or industry today. This week, we'll get an overview of what machine learning is, what it can be used for and what its limits are. Without worrying too much about what goes on behind the scenes, we will play around with a few classifiers in Python and test model performance using cross validation. The exercises today cover:

* Feature representation
* Model fitting
* Model evaluation
* Prediction results

[**Feedback**](http://ulfaslak.com/vent)

## Exercises

We want to predict whether a character is a hero or a villain from information that we can extract from their markup. This is a large problem that includes some data wrangling, model fitting and a bit of evaluation. Therefore the problem is split into parts.

### Part 1: Feature representation
In it's raw format, the data cannot be given to a machine learning algorithm. What we must do is extract features from the data and put them into a structured format. This is the same as what we did when we looked at a dog (the data) and extracted into a matrix whether it was fluffy, sad looking, etc. (the features). The feature we will extract here is **team alliances**.

We can represent the team alliances of each character as a row in a matrix where each column corresponds to a particular team. That should look something like this (numbers are made up):

<img src="http://ulfaslak.com/computational_analysis_of_big_data/exer_figures/example_boa.png" width="400"/>

**Note**: The following exercises relies on the dataset you produced in Ex. 3.1.2 (character markup stored on your computer). If you didn't manage to produce the dataset [use mine](https://github.com/ulfaslak/practical_data_science/tree/master/data) and go back and complete it at a later time so that you get the most out of this session.

> **Ex.4.1.1 (solved)**: Write a function called `get_alliances` that takes the name of a character and returns a list of teams that the character is allied with. Print the alliances of Iron Man.
>
>*Hint: There is a place in the character markup where you'll find the team affiliations and you want to write some code that can extract that. The affiliations are not always listed the same way so you should probably try to eyeball some of the documents to get a feel for how the formatting can look.*

In [1]:
import re, os

def get_alliances(char, faction=None):
    """Return list of alliances for Marvel character.
    
    Input
    -----
        char : str
            A valid character name of any faction
        faction : str
            Either 'heroes', 'villains', 'ambiguous' or None. If None, the function
            looks through the respective faction folders to figure out which faction
            `char` belongs to. The function is therefore FASTER if `faction` is provided
    
    Output
    ------
        out : list of strings
            List of alliance names
    """
    
    # If faction is not provided, figure out which faction it is by looping through
    # folders of character names
    if faction is None:
        for faction in ["heroes", "villains", "ambiguous"]:
            if char + ".txt" in os.listdir("../data/%s" % faction):
                break
    
    # Load character markup
    with open("../data/%s/%s.txt" % (faction, char)) as fp:
         markup = fp.read()

    # Get alliance field
    alliances_field = re.findall(r"\| *alliances[\w\W]+?(?=\|.+=|\}\})", markup)
    if alliances_field == []:
        return []

    # Extract teams from alliance field
    return [
        t[2:-1]
        for t in re.findall(r"\[\[.+?[\]\|\#]", alliances_field[0][10:])
        if not 'List of' in t
    ]

In [2]:
get_alliances("Iron Man")

['Avengers (comics)',
 'United States Department of Defense',
 'Force Works',
 'The New Avengers (comics)',
 'Guardians of the Galaxy (2008 team)',
 'Illuminati (comics)',
 'The Mighty Avengers',
 'S.H.I.E.L.D.',
 'Stark Industries',
 'Stark Resilient',
 'Thunderbolts (comics)']

> **Ex.4.1.2**: Produce a **sorted** list called `all_teams` that contains all teams in the entire Marvel universe. Print the first 10.

> **Ex.4.1.3**: Write a function that takes the name of a character and returns a vector representation of the team alliances for that character. The length of the returned list will be `len(all_teams)` and have 0s in the places corresponding to teams that the character is not on, and 1s in the places corresponding to teams the character is on. It may look something like:
>
>        [0, 0, 0, 0, 0, 1, 0, 0, ..., 0]
>
> Print the sum of the list returned when the input to the function is "Iron Man". Show that this sum is equal to `len(get_alliances("Iron Man"))`.

> **Ex. 4.1.4**: Vectorize and save your data:
1. Create a feature matrix (a "team alliance matrix") similar in form to the matrix illustrated at the top of this notebook, in that every row corresponds to a character and every column a team. In a seperate *target* array: store whether characters are villains (denote by 0), heros (denote by 1) or ambiguous (denote by 2). **Do not include characters that have no team alliances**. The number of rows in your feature matrix should equal the length of the target array. **Print their shapes to show that they match each other.**
2. Create a new `pandas.DataFrame` called `data` that stores all this data. Rows should be indexed with character names, columns should be indexed team names and the rightmost column should be the target array. [Here is what mine looks like](https://ulfaslak.com/computational_analysis_of_big_data/exer_figures/example_4.1.4.2.png). Save this dataframe as a CSV file to a local folder. Load it again and **show that it looks right** (don't use `print`!), you will need it in the coming weeks.

### Part 2: Model fitting

> **Ex. 4.2.1**: Take the feature matrix and target array that you created in Ex. 4.1.4 and **remove the rows that correspond to ambiguous characters**. Now, train a classifier for predicting hero or villain on all of this data and test its accuracy on the same data you trained on.
>
>* If your team alliance matrix is `X_ta` and your target array is `y_ta` you can do this by instantiating a model like:
>
>        from sklearn.naive_bayes import BernoulliNB
>        model = BernoulliNB()
>        model.fit(X_ta, y_ta)  # <--- This is the training step
>        
> *Note:* The `BernoulliNB` is a version of the Naive Bayes classifier which associates certain features with labels and asks what the probability of a label for a data point is given its features. You are free to use any other classifier if you want. Popular ones are trees, random forests, support vector machines, feed forward neural networks, logistic regression, and the list goes on. With `sklearn`, they are just as easy to employ as the `BernoulliNB` classifier.
>
>
>1. Test the accuracy of your model. Report the *training accuracy* of your model—that is testing on the same data that you trained the model on. 
2. Which *baseline accuracy* does this model have to compete with? In other words, what accuracy would you get if you always predicted the majority class?
>
>3. Report the precision, recall and F1 scores, with respect to the **minority class**.
>
> *Hint: `model.predict(X_ta)` will give you predictions for each datapoint.*

### Part 3: Model evaluation

> **Ex. 4.3.1**: Investigate how well your model generalizes. You may have noticed that the performance seemed a little too good to be true in Ex 4.2.1.
1. Why did you get such a high accuracy in the previous exercise?
2. Split your data into a test and training set of equal size. Train the model only on the training set and report its accuracy and F1 score (for minority class) on both the training and test sets.
3. Comment on the difference you observe.
>
> *Hint: Watch out for unbalanced class proportions! You may want to randomly reorder the rows of your datapoints and target labels so your training and test sets have the same amount of heroes and villains.*

> **Ex. 4.3.2**: Implement cross validation using the [`sklearn.model_selection.KFold`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) method. The performance of a classifier is strongly dependent on the amount of data it is trained on. In Ex. 4.3.1 you train it on only half of the data and test it on the other half. If you rerun that code multiple times, with random 50/50 partitions, you are gonna see a lot of uncertainty in performance. Cross validation solves this problem by training on a larger subset of the data and testing on a smaller one, and taking the average performance over K-folds of this process.
1. Explain how the `KFold` function (imported from `sklearn.model_selection`) works.
2. Using `KFold`, implement cross validation over $K=10$ folds. For each fold you must record the training and test accuracies. In the end, visualize the distributions of test- and training accuracy as histograms in the same plot. Why are training accuracies consistently higher than testing accuracies (comment)?.

### Part 4: Predicting good vs. evil

>**Ex. 4.4.1**: Let's put our classifier to use!
* Retrain your model on all of your data (still only heroes and villains).
* Get a team alliance representation of the ambiguous characters
* Use the model the estimate the probability that each character is a villain (let's call this *villainness*). You can use the `.predict_proba` method on the model to get probability estimates rather than class assignments. It produces an array with two columns, where the 1st column is the probability of class 1 (which is heroes).
* **Visualize the "heroness" distribution for all ambiguous characters**. Comment on the result.